# Load the modules needed 

In [1]:
# Basic Function Modules
import sys
import os
import glob
from imp import reload
import csv
import threading
from pathlib import Path
import warnings
#Data Analysis Modules
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Image Detection Modules
import skimage.measure as measure
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
#AutomatedCE Modules
import testing.inputs  
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO
from L4 import Focus
from L4 import AutomatedControl
from L4 import Trajectory
from L4.image_util import ImageSaver
from L3 import SystemsBuilder
import testing.te300_auto_utils as auto_utils
from L4 import image_util
from L4 import FileIO


# Reload (when changing aspects in the following modules)
reload(Focus)
reload(DisplayWindows)
reload(testing.inputs)
import warnings
warnings.simplefilter("ignore", UserWarning)
# Make the plots pop up in a separate window
%matplotlib qt


new directory is: C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE
C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\MicroControlServer.py


## Step 1. Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

If you get an error, you must restart the kernel before re-running. 

In [2]:
#CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\config\TE300.cfg"
#CONFIG_PATH = r"D:\Scripts\CEInstrument\AutomatedCE\config\Test-System.cfg"
reload(SystemsBuilder)

# Create the system object
ce_system = SystemsBuilder.CESystem()
# Read the config file
ce_system.load_config(CONFIG_PATH)
# Open the controllers and run the startup functions
ce_system.open_controllers()
ce_system.startup_utilities()

# Adjust the poorly wired RGB lighting
ce_system.inlet_rgb.turn_on_channel('B')
ce_system.inlet_rgb.turn_off_channel('R')

# Adjust the excitation wheel start position
ce_system.filter_wheel.set_channel(1)
ce_system.camera.set_exposure(100)

pulses=0

daq
newwave
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x0000015BFF4D3760>, 'xy_stage': <L2.XYControl.PriorXY object at 0x0000015BFF4D37C0>, 'objective': <L2.ZControl.PriorZ object at 0x0000015BFF4D3820>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x0000015BFF4D3850>, 'inlet_rgb': <L2.LightControl.RGBArduino object at 0x0000015BFF4D37F0>, 'inlet_z': <L2.ZControl.KinesisZ object at 0x0000015BFF4D38E0>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x0000015BFF4D38B0>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x0000015BFF4D3940>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x0000015BFF4D3D60>, 'lysis_laser': <L2.LaserControl.NewWaveBNC object at 0x0000015BFF4D3FD0>, 'excitation_wheel': <L2.FilterWheelControl.LumencorFilter object at 0x0000015BFF4D3FA0>, 'filter_wheel': <L2.FilterWheelControl.PriorFilter object at 0x0000015BFF4D3EE0>}
OPENING:  ard1 <L1.Controllers.ArduinoController object at 0x0000015BF

In [7]:
# Turn off LED
ce_system.inlet_rgb.turn_off_channel('B')


'R:Off, G:Off, B:Off'

In [8]:
# Turn on LED
ce_system.inlet_rgb.turn_on_channel('B')

'R:Off, G:Off, B:On'

In [3]:
ce_system.lysis_laser

In [106]:
pulses=0

# Step 2. Prepare the Capillary for Electrophoresis

We need to prep the capillary for CE. This involves rinsing the capillary surface and removing contaminants as well as re-establsihing a deprotonated silanol layer. We also need to align the capillary with detection objective. 

### Step 2.1 Rinse the capillary (Repeatable)

Rinse the capillary according to your method. A good place to start is 10 min with HCl, 30 min with NaOH, 2 min with water. If this is your first rinse of the day,rinse with an empty vial first to make sure there are no leaks on the outlet side of the apparatus. 

In [15]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [16]:
# Stop Rinse 
ce_system.outlet_pressure.release()

### Step 2.2 Align the capillary with fluorescein. 

Place a vial with 125 nM fluorescein in the outlet. Once the pressure is rinsing adjust the capillary position using the two set screws as well as the adjustable objective collar until the signal is maximized for the detector being used. (PMT gain voltage should be set to 250 mV, or the lowest setting with appreciable gain). 

In [18]:
# Run the CE View
reload(DisplayWindows)

# Create a CE Display
ce_disp = DisplayWindows.PLTCEDisplay(ce_system)
ce_disp.show()
ce_system.detector.start()
ce_disp.start_live_view()

In [ ]:
resp = get_yes_no("Is fluorescein in the outlet and ready to be rinsed with?")

if resp:
    ce_system.outlet_pressure.rinse_pressure()
    print("Adjust the capillary inlet while maximizing the signal")
if not resp:
    print("Please insert the fluorscein in the outlet, then re-run this cell.")

When finished. Record the maximum signal, along with the detector used, its gain setting (or bias when using SiPM), and the fluorescein concentration in the calibration log (located on the desktop). 

Run the next cell to stop the pressure rinse. 

In [ ]:
ce_system.outlet_pressure.release()

### Step 2.2 Post Fluorescein Rinse

Take a vial and add close to 4 mL of H20. Insert it into the outlet and then remove it. This removes any residual Fluorescein from the capillary, electrode, or other surfaces.  

Repeat with a second vial, with liquid slightly higher or the same height. 

Finally switch with separation buffer and rinse for 10 min. 

In [ ]:
resp = get_yes_no("Have you done the pre-fluorescein rinses?")
if not resp:
    print("Please follow instructions in step 2.2")
else:
    resp = get_yes_no("Is the separation buffer placed in the outlet?")
    if resp:
        ce_system.outlet_pressure.rinse_pressure()
        print("Run the next cell when finished")
    else:
        print("Please place the separation buffer in the outlet before rinsing.")

In [ ]:
ce_system.outlet_pressure.release()

# Step 3. Calibrate the Capillary Inlet

The method for calibration depends on the z-stage used to raise the capillary inlet. Select the correct route for the hardware employed. 

## Thorlabs Kinesis Z Stage

### Step 3.1 Lower the Capillary (thorlabs)
Move the capillary to a safe location where it will not be crushed if the stage lowers too far. Using the controls on the Z-stage itself, lower the capillary till as far as it will go. 

In [24]:
resp = get_yes_no("Have you lowered the Kinesis Z stage to its lowest position? (y/n):  ")
if not resp:
    print("Please make sure the Kinesis stage is lowered to its lowest position before continuing")
else:
    print("Proceed to the next cell.")
    

Have you lowered the Kinesis Z stage to its lowest position? (y/n):   y
Proceed to the next cell.


### Step 3.2 Home the Capillary (thorlabs)

Thorlabs needs to register the home position. In the next cell we will call the go_home command to home the capillary. 

In [25]:
ce_system.inlet_z.homing()

### Step 3.3 Adjust Capillary Position (all stages)

Adjust the capillary position by sliding the detection apparatus up or down the optical support post. The capillary should be level with the bottom of the template for single-cell assays, or the top-of the template for well-based assays. 

*Aligning to the top of the template is a minor safety feature to prevent the user from sending the capillary to Zero position unexpetedly and crashing the capillary against the bottom of the template. However, capillary needs to be able to reach at least the glass slide of the cell reservoir for single-cell assays*

In [26]:
resp = get_yes_no("Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  ")
if not resp:
    print("Please make sure the capillary is moved to the correct height with respect to the template before continuing")
else:
    print("Proceed to the next cell")

Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):   
Please enter y or n as you answer
Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):   y
Proceed to the next cell


### Step 3.4 Raise the capillary to a safe height

The next cell will raise the capillary height to a location where it ***should*** be safe for any XY movements. Verify the capillary will not crash at this new height. 

In [27]:
target_z = get_float("Please enter a safe height to move the stage to: ")
# Move the capillary up and wait for it to stop
ce_system.inlet_z.set_z(target_z)
ce_system.inlet_z.wait_for_target(target_z)

resp = get_yes_no("Is the capillary at a safe location?: ")
if resp:
    print("Continue to next step")
else:
    print("Please raise the capillary to an appropriate height")

Please enter a safe height to move the stage to:  24
Is the capillary at a safe location?:  y
Continue to next step


# Step 4. Home the XY Stage

This next step provides the calibration for the XY axis for the XY stage and capillary inlet. 


### Step 4.1 (TE300, Prior)

The TE300 does not have the encoders wired to tell it where absolute 0,0 is. Thus, we need to tell the microcontroller where 0,0 is for each template. The home position before any offset, will be when the objective is in the top-right corner (if looking down at it). 

1. Move the objective to the upper right corner of the template. 
2. Align the capillary over the well or the objective


In [28]:
resp = get_yes_no("Is the capillary aligned with the objective and home well?")
if resp:
    ce_system.xy_stage.set_home()
    print("Home set at this position.")
    print("Continue to next step")
else:
    print("Please raise align the capillary with the objective")   

Is the capillary aligned with the objective and home well? y
Home set at this position.
Continue to next step


# Step 5 Verify your calibration

Steps 3-4 calibrate the capillaries position with the microscope's position. Now we need to verify that our alignment is correct. To do this we will move the capillary to a well and make sure that it is centered over the well. Then we will lower the capillary to a height specified by the method to make sure the capillary can reach the liquid. 

### Step 5.1. Load the Template File

The template file tells the program where to move the XY stage for each well as well as how high to raise the capillary inlet during a move. Change the variable TEMPLATE_PATH to the absolute path where the template.txt file resides. 

In [29]:
TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\var\TE300CellTemplate.txt"

reload(AutomatedControl)
reload(Trajectory)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)

### Step 5.2a Raise the capillary to a safe height 

In [30]:
z = get_float("Set the capillary to a safe travel height: ")
ce_system.inlet_z.set_z(z)
print("Proceed to next cell")

Set the capillary to a safe travel height:  25
Proceed to next cell


### Step 5.2b Center the Capillary over a well

In [31]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): w_a1
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.


### Step 5.2c Adjusting the capillary center software offset.
**(you can run this after aligning the cap to the objective for better centering over vials)**
Occasionally, when the capillary is aligned over the objective, it will no longer be aligned over the vials. The next commands allow you account for this slight shift. Unlike the calibration steps above, this step allows you to move the XY-stage to center the well and keep the capillary stationary. This is most important for when the electrode or capillary look as if they may hit the vial walls during the run. 

In [33]:
# If you had to create a new 'auto_program' object, run this command to run the xy_offset from earlier.
try:
    auto_program.offset = xy_offset
except:
    print("XY Offset not set")

XY Offset not set


In [34]:
# Run this command to create a new XY_offset 
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
auto_program.move_to_well(name)
resp = get_yes_no("Move the capillary using the xxy stage joystick, or the z-stage base so that capillary is centered over this well. \n Is the capillary centered?")
if resp: 
    xy_offset = auto_program.offset_template(well.xy)

Please enter the well name to move to (press enter to see a list): w_a2
Move the capillary using the xxy stage joystick, or the z-stage base so that capillary is centered over this well. 
 Is the capillary centered? y
Offset is: [0.9650000000000034, -1.398142857142858]


### Step 5.3 Lower the capillary to the height described in the method file


In [37]:
height_z = get_float("What height should the capillary be lowered to? (in mm, use inlet height from methods file)")
prev_height = ce_system.inlet_z.read_z()
ce_system.inlet_z.set_z(height_z)
resp = get_yes_no("Is the capillary touching the liquid in this well? ")
if not resp:
    print("Adjust the height of the capillary manually (and carefully!) so that the capillary is touching the liquid")
else:
    print("Calibration Finished")
ce_system.inlet_z.set_z(prev_height)

What height should the capillary be lowered to? (in mm, use inlet height from methods file) 5
Is the capillary touching the liquid in this well?  y
Calibration Finished


### Optional> 
Read out the height you are currently at...

In [46]:
print("You are at {} mm".format(ce_system.inlet_z.read_z()))

You are at 10.71127 mm


In [ ]:
resp = get_yes_no("Would you like to lower the capillary?")
if resp:
    resp = get_float("What would you like to lower it to?")
    ce_system.inlet_z.set_z(resp)
else:
    pass

# Step 6. Load a method

### NOTE: Step 5.1 must be run before this 

Load a method file. Verify your method file in excel and make sure the run is set up as intended. You can add multiple files to the same auto_program

If you need to reset the methods (remove and start over) run the next cell

In [38]:
# Clears the methods to be added to the run protocol
auto_program.methods=[]

In [39]:
# Put the filepath to your method here:
file_to_add = r"C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\var\Luke\Fluoresceinstandard.txt"
# Reads the method, adds it to our list of methods to run
auto_program.add_method(file_to_add)

WHASSUZ ['inject\tw_a2, w_a3, w_a4, w_a5\t21 cm\t5 mm\t0\t0\t10 kV\t3 s\t\t0', '50 nm sf s1pf 324 mv\tw_a1\t21 cm\t5 mm\t0\t0\t20 kV\t160 s\t\t1']
ADDING
ADDING


In [40]:
# Set the number of repetitions to run each method
auto_program.repetitions=12

# Setup Powerpoint Output

In [110]:
reload(auto_utils)
reload(image_util)
reload(FileIO)
SlideSingleCell = FileIO.SlideSingleCell


AttributeError: module 'L4.FileIO' has no attribute 'SlideSingleCell'

Exception in thread Thread-111581:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L2\CameraControl.py", line 240, in _sequence_update
    img = self.controller.send_command(self.controller.core.get_last_image)
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\Controllers.py", line 232, in send_command
    raise e
  File "C:\Users\NikonTE300CE\Desktop\Barracuda_Updated\AutomatedCE\L1\Controllers.py", line 223, in send_command
    ans = command(*args)
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\site-packages\pycromanager\core.py", line 399, in <lambda>
    ): instance._translate_call(signatures_list, args)
  File "C:\Users\NikonTE300CE\miniconda3\envs\

In [ ]:
# Enter power point file name here
file_name = 'SingleCellData'

# Ask 
resp = get_yes_no("Do you want to save the old presentation (if applicable) and start a new record?\n"
                  "Previous Presentations will not be overwritten. \n" 
                  "Slides will be saved to the Data Folder")
if resp:
    cell_notes = ""
    file_name = FileIO.get_data_filename(file_name+'.pptx', auto_program.data_dir, extension=".pptx")
    slide_show = SlideSingleCell(file_name)
    last_lysis = None

# Automated Run Start and Stop Commands

In [41]:
# Start the run
auto_program.start_run()

After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  160


In [51]:
# Stop the Run
auto_program.stop_run()
auto_program.continue_event.set()

ERROR:root:Error while moving during System Move. Aborting
ERROR:root:Exiting the run...
Exception in thread Thread-302249:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
Thorlabs.MotionControl.DeviceManagerCLI.MoveTimeoutException: Move failed to complete in time
   at Thorlabs.MotionControl.GenericMotorCLI.GenericMotorCLI.Wait(Int32 waitTimeout)
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.MoveTo(Decimal position, Int32 waitTimeout)


In [84]:
# Run to continue past a wait step
auto_program.continue_event.set()

After special True
Time to wait is  3
After special True
Time to wait is  120


# Displays

In [42]:
# CE Display
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()

# Uncomment the following to reset the detector (start gathering data)
#ce_system.detector.start()

After special True
Time to wait is  3
After special True
Time to wait is  160


After special True
Time to wait is  3


After special True
Time to wait is  160
XY Errr ['']
After special True
Time to wait is  3


XY Errr ['']
After special True
Time to wait is  160
XY Errr ['']
After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  160


After special True
Time to wait is  3


...

XY Errr ['']
After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  160


XY Errr ['']
After special True
Time to wait is  3
XY Errr ['']
After special True
Time to wait is  160
After special True
Time to wait is  3
After special True
Time to wait is  160


After special True
Time to wait is  3


After special True
Time to wait is  160
XY Errr ['']
After special True
Time to wait is  3


XY Errr ['']
After special True
Time to wait is  160


In [46]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
cam.scalar=1
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


# SET DISTANCE BETWEEN OBJECTIVE AND CAPILLARY

In [47]:
# If this is not 0, make sure the objective won't crash!
ce_system.objective.read_z()

-0.0002

In [45]:
# Go to Zero at the start of the day
ce_system.objective.set_z(0)

Now bring cells into focus, by using the coarse objective adjustment! Do not change the coarse adjustment from here on out.

In [79]:
# Move the objective to the cell focus, and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.set_rel_z(.05)

In [78]:
# Lower Capillary Inlet into Cell Chamber units in mm
ce_system.inlet_z.set_rel_z(0.005)

In [80]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()
cap_difference = obj_height - cap_height
with open('save_me','w') as fout:
    fout.write(f'cap_difference,{cap_difference}')

In [81]:
# if you needed to restart you can load the earlier difference
with open('save_me','r') as fin:
    cap_difference= float(fin.readline().split(',')[1])
    

In [82]:
ce_system.inlet_z.read_z()

14.168074

In [83]:
ce_system.inlet_z.set_z(30)

In [84]:
ce_system.objective.set_rel_z(-0.05)

# Take Fluorescent Images

This is for the NikonTE300 Microscope. 

*light_channel* corresponds to the lumencor light. Please enter the lowercase name of light to show:
'red', 'blue', 'cyan', etc...

*fluor_channel* corresponds to the filter wheel position for the emission filter. 

GFP Fluoresence settings:
light_channel = 'cyan'
fluor_channel = 4


In [6]:
fluor_channel=4
light_channel='cyan'
bins=2
ce_system.excitation_wheel.set_intensity(['cyan'],1)
auto_utils.pre_fluoresence(ce_system)
img = ce_system.camera.snap()
auto_utils.post_fluoresence(ce_system)



HEYYOO


Exception in thread Thread-135:
Traceback (most recent call last):
  File "D:\Software\MiniConda\envs\CEpy3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\Software\MiniConda\envs\CEpy3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "D:\Scripts\CEInstrument\AutomatedCE\L2\CameraControl.py", line 241, in _sequence_update
    img = self.controller.send_command(self.controller.core.get_last_image)
  File "D:\Scripts\CEInstrument\AutomatedCE\L1\Controllers.py", line 232, in send_command
    raise e
  File "D:\Scripts\CEInstrument\AutomatedCE\L1\Controllers.py", line 223, in send_command
    ans = command(*args)
  File "D:\Software\MiniConda\envs\CEpy3\lib\site-packages\pycromanager\core.py", line 399, in <lambda>
    ): instance._translate_call(signatures_list, args)
  File "D:\Software\MiniConda\envs\CEpy3\lib\site-packages\pycromanager\core.py", line 519, in _translate_call
    return self._deserialize(self._socket.rece

In [ ]:
# Show the image you just took 
fig, ax = plt.subplots()
ax.imshow(img)

In [71]:
# Show the histogram of the image you just took 
fig,ax = plt.subplots()
ax.hist(img.flatten())

(array([ 3718., 12182.,  3998.,  1532.,   633.,   326.,   129.,    55.,
           33.,    14.]),
 array([192. , 206.2, 220.4, 234.6, 248.8, 263. , 277.2, 291.4, 305.6,
        319.8, 334. ]),
 <BarContainer object of 10 artists>)

# SINGLE CELL 

In [85]:
# Run this box, when you need to raise the objective up, following a single cell. 
resp = get_yes_no("you want to raise the objective?")
if resp:
    ce_system.objective.set_rel_z(5)

you want to raise the objective? n


In [58]:
# Use these two commands to record, or go to previous locations on the glass slide. 
#xy = ce_system.xy_stage.read_xy()
#ce_system.xy_stage.set_xy(xy)

In [86]:
# Record lysis images? 
record_lysis = True
lysis_dir = r"C:\Users\NikonTE300CE\Desktop\Luke\Lysis\images"

### Find your cells of interest, lower the capillary 
Find cells of interest and focus them before running the next command. Adjust Drift if necessary. 
Drift is how much of an offset on the capillary Z. 

In [109]:
ims = image_util.ImageSaver(ce_system,folder_prefix='lysis', data_folder=lysis_dir )

def save_slides(slide_show, *args):
    warnings.filterwarnings("ignore")
    slide_show.add_slide(*args)
    slide_show.save_presentation()
    warnings.filterwarnings("default")

resp=True

if resp:
    # on reruns, you just need to run this command when you have the cells in focus
    resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
    drift = +0.0
    if resp:
        obj_height = ce_system.objective.read_z()
        print(obj_height)
        cap_z = auto_utils.get_cap_height(obj_height, cap_difference)
        z = ce_system.inlet_z.read_z()
        rel_z = cap_z-z
        rel_z+=drift
        ce_system.inlet_z.set_rel_z(rel_z)

        lysis_attempts = 0 
        if record_lysis:
            ims.add_callback()
        
        if auto_program.last_path is not None:
            print("Saving Presentation... this may take 15-20 s")
            file_path = str(auto_program.last_path)
            cell_notes = f'EK Injection: {voltage_level} kV, {injection_time} s, Pulses required: {pulses} \n' + cell_notes
            threading.Thread(target=save_slides, args=(slide_show, file_path,last_lysis, None, cell_notes)).start()
            auto_program.last_path=None
            pulses=0
    else:
        print("Bring cells into focus.")
        

are the cells in focus? (You may crush the capillary if not...) y
0.1431
Saving Presentation... this may take 15-20 s


NameError: name 'slide_show' is not defined

Exception in thread Thread-122029:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
Thorlabs.MotionControl.DeviceManagerCLI.MoveToInvalidPositionException: 49125264: Action will result in an invalid position
   at Thorlabs.MotionControl.DeviceManagerCLI.ThorlabsGenericCoreDeviceCLI.RaiseDeviceException(CDeviceException* exception)
   at Thorlabs.MotionControl.GenericMotorCLI.GenericMotorCLI.RaiseDeviceException(CDeviceException* exception)
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.MoveStep_internal(MotorDirection direction, Action`1 callback)
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.MoveJog(MotorDirection direction, Int32 waitTimeout)


Check the capillary drift here. The first of the two cells will let you move the objective. Do so to investigate the capillary p ositoin. 

In [101]:
# Raise the objective for optimal plasma formation
# Optional, uncomment if needed
ce_system.objective.set_rel_z(0.002)

In [103]:
# Lower the capillary (ADJUSTMENT)
# If desired, add this to your drift to keep the capillary in the correct distance above the cell focus. 
ce_system.inlet_z.set_rel_z(0.025)

### Lyse the Cell and load the lysate into the capillary 

In [104]:
# Start the Single Cell Injection

# General Injection Parameters
injection_time = 3 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 10 # in kilovolts
delay = 1 # inseconds



import threading
#pulses+=1
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
        
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.outlet_pressure.release()
    ce_system.high_voltage.start()
    start_time = time.time()
    time.sleep(0.1)
    ce_system.lysis_laser.laser_standby()
    ce_system.lysis_laser.laser_fire()
    
    first=False
    while time.time()-start_time < injection_time:
        time.sleep(0.1)
        if time.time()-start_time >10 and first:
            ce_system.lysis_laser.laser_fire()
            first=False
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    ce_system.outlet_pressure.seal()
    

#lysis_attempts+=1
threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()
#threading.Thread(target=fire_me, args=arg).start()

Fire
Finished 3.025714635848999


In [233]:
# Run this to push out ECB buffer if you want to redo injections
ce_system.outlet_pressure.rinse_pressure()
time.sleep(10)
ce_system.outlet_pressure.release()

In [100]:
#Move the capillary up to see better
ce_system.inlet_z.set_rel_z(0.5)

### Continue to the separation (or whatever the next step in the method is) 

In [57]:
# Continue the Method
def go_ahead():
    global last_lysis 
    ce_system.objective.set_rel_z(0.05)
    time.sleep(2)
    ce_system.inlet_z.set_rel_z(3)
    ce_system.objective.set_rel_z(-0.05)
    time.sleep(2)
    ce_system.objective.set_rel_z(-5)
    if not auto_program.continue_event.is_set():
        
        auto_program.continue_event.set()
    else:
        auto_program.start_run()
        
    if record_lysis:
        ims.remove_callback()
        
        last_lysis = ims.save_gif('Lysis', None, scale=0.25)
    
threading.Thread(target=go_ahead).start()

Exception in thread Thread-338:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-57-01bd0674e48e>", line 16, in go_ahead
    if record_lysis:
NameError: name 'record_lysis' is not defined



After special True
Time to wait is  2


### Add any notes about the cell or injection that you wish to record here. 

If you are collecting NA it is a good idea to record which pcr tube you are collecting this into. 
ie (Strip A, Well 1) 

In [108]:
cell_notes = 'practice, 6th of the day'

# misc functions

In [217]:
# Record the previous path as the last path, this should be run when restarting the auto_program object
auto_program.last_path = file_path
file_path

'C:\\Users\\NikonTE300CE\\Documents\\Data\\MiaPaCa 20 mM I_00055.csv'

In [440]:
# Run this when you have finished the last repetition
file_path = r"C:\Users\NikonTE300CE\Documents\Data\MiaPaCa 20 mM I_00071.csv" #str(auto_program.last_path)
cell_notes = f'EK Injection: {voltage_level} kV, {injection_time} s, Pulses required: {pulses} \n' + cell_notes
threading.Thread(target=save_slides, args=(slide_show, file_path,last_lysis, None, cell_notes)).start()
auto_program.last_path=None
pulses=0

0.13137462749445225
0.039348088551829634
0.03305663045761187
0.0024370335196449077
0.00210813739937042
0.0022232124694871484
0.0019132362323810905
0.0002587032163300099


In [354]:
# Save the presentation 
slide_show.save_presentation()

Exception in thread Thread-185947:
Traceback (most recent call last):
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
Thorlabs.MotionControl.DeviceManagerCLI.MoveToInvalidPositionException: Cannot move to requested position
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.VerifyDeviceMovement(Int32 functionDepth, Decimal position, PositionType positionType)
   at Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor.GenericAdvancedMotorCLI.MoveTo(Decimal position, Int32 waitTimeout)

ERROR:root:Error while moving during Inlet Move Down. Aborting
ERROR:root:Exiting the run...


In [45]:
ce_system.outlet_pressure.rinse_pressure()

In [46]:
ce_system.outlet_pressure.release()

In [307]:
ce_system.objective.set_rel_z(-5)

After special True
Time to wait is  120
After special True
Time to wait is  45


After special True
Time to wait is  45


C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:277: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))


After special True
Time to wait is  45


In [456]:
ce_system.outlet_pressure.seal()

In [53]:
ce_system.inlet_z.read_z()

0.8488835

In [37]:
ce_system.inlet_z.set_z(24)